# 🚀 Super Gnosis ML Training Pipeline Demo

This notebook demonstrates the complete ML training pipeline with FREE data sources.

## What You'll Learn
1. Fetch FREE market data (yfinance + Yahoo Options)
2. Generate ML labels (direction, magnitude, volatility)
3. Engineer 141 features from engines
4. Train multi-task models (LightGBM)
5. Make predictions with confidence calibration

## Cost: $0/month
All data sources are FREE. No paid subscriptions required!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DGator86/V2---Gnosis/blob/main/notebooks/01_ML_Training_Pipeline_Demo.ipynb)

## 📦 Setup & Installation

In [ ]:
# Install Super Gnosis (uncomment if running in Colab)
# !pip install git+https://github.com/DGator86/V2---Gnosis.git

# Or install from PyPI (once published)
# !pip install super-gnosis

# For local development
import sys
sys.path.append('/home/user/webapp')

import warnings
warnings.filterwarnings('ignore')

## 1️⃣ Fetch FREE Market Data

In [ ]:
from engines.inputs.yfinance_adapter import YFinanceAdapter
import polars as pl

# Initialize FREE data adapter
adapter = YFinanceAdapter()

# Fetch 60 days of SPY data (5-minute bars)
print("📊 Fetching SPY data from Yahoo Finance (FREE)...")
df_ohlcv = adapter.fetch_history(
    symbol="SPY",
    period="60d",
    interval="5m"
)

print(f"✅ Fetched {len(df_ohlcv)} bars")
print(f"\nFirst 5 rows:")
df_ohlcv.head()

In [ ]:
# Get market regime data (VIX, SPX)
regime = adapter.fetch_market_regime_data()

print(f"🌡️  Market Regime:")
print(f"   VIX: {regime['vix']:.2f}")
print(f"   SPX: {regime['spx']:.2f}")

## 2️⃣ Generate ML Labels

In [ ]:
from ml.labels.generator import LabelGenerator, LabelConfig

# Configure label generation
config = LabelConfig(
    horizons=[5, 15, 60],  # 5min, 15min, 1hour
    magnitude_thresholds=[0.001, 0.003],  # Small/medium/large moves
    volatility_window=20
)

# Generate labels
print("🏷️  Generating ML labels...")
generator = LabelGenerator(config)
df_with_labels = generator.generate(df_ohlcv, price_col="close")

print(f"✅ Generated labels for {len(df_with_labels)} samples")
print(f"\nLabel columns:")
label_cols = [col for col in df_with_labels.columns if 'label_' in col or 'fwd_' in col]
print(label_cols)

# Show sample
df_with_labels.select(['timestamp', 'close'] + label_cols[:6]).tail(5)

## 3️⃣ Engineer Features

In production, features come from the 3 engines (Hedge, Liquidity, Sentiment).
For this demo, we'll use technical indicators.

In [ ]:
from ml.features.technical import TechnicalIndicators
from ml.features.ta_indicators import TAIndicators

# Add technical indicators
print("📈 Adding technical indicators...")
tech = TechnicalIndicators()
df_with_features = tech.add_all_indicators(df_with_labels)

# Add 130+ indicators from ta library
ta = TAIndicators()
df_with_features = ta.add_all_indicators(df_with_features)

feature_cols = [col for col in df_with_features.columns 
                if col not in df_ohlcv.columns and 'label_' not in col]

print(f"✅ Added {len(feature_cols)} features")
print(f"\nSample features:")
print(feature_cols[:20])

## 4️⃣ Prepare Dataset

In [ ]:
from ml.dataset.builder import MLDataset

# Create dataset
print("📦 Creating ML dataset...")
dataset = MLDataset(
    df=df_with_features,
    feature_cols=feature_cols,
    label_cols=label_cols,
    test_size=0.2,
    valid_size=0.1
)

# Split data
train, valid, test = dataset.split_temporal()

print(f"\n📊 Dataset sizes:")
print(f"   Train: {len(train)} samples")
print(f"   Valid: {len(valid)} samples")
print(f"   Test:  {len(test)} samples")

## 5️⃣ Train Multi-Task Models

In [ ]:
from ml.trainer.lightgbm_trainer import LightGBMTrainer, LightGBMConfig
import numpy as np

# Configure trainer
config = LightGBMConfig(
    num_leaves=31,
    max_depth=6,
    learning_rate=0.05,
    n_estimators=100,
    early_stopping_rounds=10
)

trainer = LightGBMTrainer(config)

# Train for 5-minute horizon
horizon = 5
print(f"\n🎯 Training models for {horizon}-minute horizon...")

# Prepare training data
X_train = train.select(feature_cols).to_pandas()
X_valid = valid.select(feature_cols).to_pandas()

# Get labels
y_direction_train = train.select(f'label_direction_{horizon}m').to_pandas().values.ravel()
y_direction_valid = valid.select(f'label_direction_{horizon}m').to_pandas().values.ravel()

# Train direction model
print("\n📈 Training direction classifier (±1)...")
direction_model = trainer.train_direction_model(
    X_train=X_train,
    y_train=y_direction_train,
    X_valid=X_valid,
    y_valid=y_direction_valid
)

print("✅ Direction model trained!")
print(f"\nTop 10 important features:")
feature_importance = direction_model.feature_importance(importance_type='gain')
top_features = sorted(zip(feature_cols, feature_importance), 
                     key=lambda x: x[1], reverse=True)[:10]
for feat, imp in top_features:
    print(f"   {feat}: {imp:.0f}")

## 6️⃣ Make Predictions

In [ ]:
# Predict on test set
X_test = test.select(feature_cols).to_pandas()
y_test = test.select(f'label_direction_{horizon}m').to_pandas().values.ravel()

# Get predictions
predictions = direction_model.predict(X_test)
probabilities = direction_model.predict_proba(X_test)

# Calculate accuracy
accuracy = (predictions == y_test).mean()
print(f"\n🎯 Test Accuracy: {accuracy:.2%}")

# Show sample predictions
import pandas as pd
results = pd.DataFrame({
    'actual': y_test[:10],
    'predicted': predictions[:10],
    'prob_down': probabilities[:10, 0],
    'prob_up': probabilities[:10, 1]
})

print("\n📊 Sample predictions:")
results

## 7️⃣ Save Model

In [ ]:
from ml.persistence.manager import ModelManager

# Save model
manager = ModelManager(base_path="./ml_models")

model_id = manager.save_model(
    model=direction_model,
    symbol="SPY",
    horizon=horizon,
    model_type="direction",
    metrics={"accuracy": accuracy},
    feature_names=feature_cols
)

print(f"\n💾 Model saved: {model_id}")
print(f"\nModel path: ./ml_models/SPY/direction/{horizon}m/")

## 8️⃣ Use in Production

In [ ]:
from ml.agents.ml_agent import MLAgent

# Initialize ML Agent
ml_agent = MLAgent(models_path="./ml_models")

# Get latest data point
latest_features = X_test.iloc[-1:]

# Make prediction
output = ml_agent.predict_direction(
    symbol="SPY",
    horizon=horizon,
    features=latest_features
)

print(f"\n🔮 Production Prediction:")
print(f"   Symbol: SPY")
print(f"   Horizon: {horizon} minutes")
print(f"   Direction: {'UP 📈' if output['direction'] > 0 else 'DOWN 📉'}")
print(f"   Confidence: {output['confidence']:.2%}")

## 🎉 Summary

You've just:
- ✅ Fetched FREE market data ($0/month)
- ✅ Generated ML labels (direction, magnitude, volatility)
- ✅ Engineered 150+ features
- ✅ Trained multi-task models (LightGBM)
- ✅ Made predictions with confidence scores
- ✅ Saved models for production use

## 🚀 Next Steps

1. **Add More Data Sources**:
   - FRED macro data
   - Options chains (Yahoo Finance)
   - Sentiment (StockTwits)

2. **Improve Features**:
   - Integrate Hedge Engine (dealer pressure)
   - Add Liquidity Engine (friction, POLR)
   - Include Sentiment Engine (fear/greed)

3. **Train More Models**:
   - Magnitude classifier (small/medium/large)
   - Volatility regressor
   - Multi-horizon ensemble

4. **Deploy to Production**:
   - Real-time data pipeline
   - Live predictions
   - Paper trading validation

## 📚 Resources

- [GitHub Repository](https://github.com/DGator86/V2---Gnosis)
- [Documentation](https://github.com/DGator86/V2---Gnosis/blob/main/ML_FEATURE_MATRIX.md)
- [FREE Data Sources](https://github.com/DGator86/V2---Gnosis/blob/main/FREE_DATA_SOURCES.md)

## 💬 Questions?

Open an issue: https://github.com/DGator86/V2---Gnosis/issues

**Happy Trading! 🚀📈**